# Introduction

In this notebook, we demonstrate the steps needed to create an IoT Edge deployable module from the regression model created in the [turbofan regression](./01-turbofan_regression_predictive_maintenance_train.ipynb) notebook. The steps we will follow are:
   1. Reload experiment and model from the Azure Machine Learning service workspace
   1. Create a scoring script
   1. Create an environment YAML file
   1. Create a container image using the model, scoring script and YAML file
   1. Deploy the container image as a web service 
   1. Test the web service to make sure the container works as expected
   1. Delete the web service
   
><font color=gray>Note: this notebook depends on the workspace, experiment and model created in the [turbofan regression](./01-turbofan_regression_predictive_maintenance_train) notebook.</font>

# Set up notebook

Please ensure that you are running this notebook under the Python 3.6 Kernel. The current kernel is show on the top of the notebook at the far right side of the file menu. If you are not running Python 3.6 you can change it in the file menu by clicking **Kernel->Change Kernel->Python 3.6**

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

## Configure workspace

Create a workspace object from the existing workspace. `Workspace.from_config()` reads the file **aml_config/.azureml/config.json** and loads the details into an object named `ws`, which is used throughout the rest of the code in this notebook.

In [ ]:
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.core.model import Model
from azureml.train.automl.run import AutoMLRun

ws = Workspace.from_config()

## Load run, experiment and model

Use the model information that we persisted in the [turbofan regression](./01-turbofan_regression_predictive_maintenance_train) noebook to load our model.

In [ ]:
import json 

#name project folder and experiment
model_data = json.load(open('./model_config.json'))

run_id = model_data['regressionRunId']
experiment_name = model_data['experimentName']
model_id = model_data['modelId']

experiment = Experiment(ws, experiment_name)
automl_run = AutoMLRun(experiment = experiment, run_id = run_id)
model = Model(ws, model_id)

## Create YAML file for the environment

The YAML file provides the information about the dependencies for the model we will deploy. 

### Get azureml versions

First we will use the run to retrieve the version of the azureml packages used to train the model. 

>Warnings about the version of the SDK not matching with the training version are expected 

In [ ]:
best_run, fitted_model = automl_run.get_output()
iteration = int(best_run.get_properties()['iteration'])
dependencies = automl_run.get_run_sdk_dependencies(iteration = iteration)

print(dependencies)

## Deploy model as a web service on Azure Container Instance

Deploy the best model we just created as web service on Azure Container Instance (ACI). We will use this web service to test that our model/container performs as expected. 

In [ ]:
script_file_name = 'score_turboban_rul.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model

inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = script_file_name)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                                memory_gb = 1, 
                                                tags = {'area': "digits", 'type': "automl_RUL"}, 
                                                description = 'test service for Edge ML RUL')

aci_service_name = 'turbofan-ml-rul-01'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
aci_service.wait_for_deployment(True)

print(aci_service_name)
print(aci_service.state)

In [ ]:
package = Model.package(
    ws,
    [model],
    inference_config,
    image_name='turbofanrul',
    image_label='latest'
)
package.wait_for_creation(
    show_output=True
)  # Or show_output=False to hide the Docker build logs.
# package.pull()
# ---
# Run the following commands using PowerShell on local client
# az acr login -n c84d9a2415e14b138f935998a6fe0015
# docker run -p 6789:5001 --name amltestrul3 25d637d844fb
# Log in to Postman
# Set the Postman client to POST
# Use the following URL: http://localhost:6789/score
# Set the Body to RAW/JSON
# Execute the following data:
# [
#     {
#         "Sensor1": 518.67,
#         "Sensor2": 641.94,
#         "Sensor3": 1581.93,
#         "Sensor4": 1396.93,
#         "Sensor5": 14.62,
#         "Sensor6": 21.58,
#         "Sensor7": 554.56,
#         "Sensor8": 2387.93,
#         "Sensor9": 9048.65,
#         "Sensor10": 1.3,
#         "Sensor11": 47.09,
#         "Sensor12": 521.89,
#         "Sensor13": 2387.94,
#         "Sensor14": 8133.48,
#         "Sensor15": 8.376,
#         "Sensor16": 0.03,
#         "Sensor17": 391,
#         "Sensor18": 2388,
#         "Sensor19": 100.0,
#         "Sensor20": 39.07,
#         "Sensor21": 23.4468
#     }
# ]

## Load test data

To save a couple of steps at this point, we serialized the test data that we loaded in the [turbofan regression](.01-turbofan_regression_predictive_maintenance_train) notebook. Here we deserialize that data to use it to test the web service.

In [ ]:
import pandas as pd
from sklearn.externals import joblib
import numpy

test_df = pd.read_csv("data/WebServiceTest.csv")

test_df.head(5)

## Predict one message at a time

Once the container/model is deployed to and Azure IoT Edge device it will receive messages one at a time. Send a few messages in that mode to make sure everything is working.

In [ ]:
import json
import pandas as pd

# reformat data as list of messages
X_message = test_df.head(5).to_dict('record')

cols_feature = ['Sensor'+str(i) for i in range(1,22)]

result_list = []

for row in X_message:
    row_features = { k: row[k] for k in cols_feature }
    row_data = json.dumps({ 'data' : [ row_features ]})
    row_result = aci_service.run(input_data=row_data)
    result_list.append(json.loads(row_result))
    
print(result_list)

## Predict message set

To make sure the model as a whole is working as expected, we send the test set in bulk to the model

In [ ]:
import json
import pandas as pd

# reformat data as list of messages
X_message = test_df.head(5).to_dict('record')

cols_feature = ['Sensor'+str(i) for i in range(1,22)]

input_data = { 'data': [] }

for row in X_message:
    row_features = { k: row[k] for k in cols_feature }
    input_data['data'].append(row_features)

input_data = json.dumps(input_data)
    
result = aci_service.run(input_data=input_data)
    
print(result)

## Delete web service

Now that we are confident that our container and model are working well, delete the web service.

In [ ]:
from azureml.core.webservice import Webservice

aci_service = Webservice(ws, aci_service_name)

aci_service.delete()